In [1]:
import networkx as nx
from main import Board, PlayerRandom

In [2]:
N = 4
board = Board(N)

In [3]:
board.play_game(players=[PlayerRandom(), PlayerRandom()])

New game.

Player 1 inserts tile ░░ from the bottom of column A.
  A B C D 
1 ________
2 ________
3 ________
4 ░░______

Player 2 places tile ██ at A2.
  A B C D 
1 ________
2 ██______
3 ________
4 ░░______

Player 1 places tile ░░ at B1.
  A B C D 
1 __░░____
2 ██______
3 ________
4 ░░______

Player 2 places tile ██ at C2.
  A B C D 
1 __░░____
2 ██__██__
3 ________
4 ░░______

Player 1 inserts tile ░░ from the top of column B.
  A B C D 
1 __░░____
2 ██░░██__
3 ________
4 ░░______

Player 2 inserts tile ██ from the left of row 4.
  A B C D 
1 __░░____
2 ██░░██__
3 ________
4 ██░░____

Player 1 inserts tile ░░ from the top of column C.
  A B C D 
1 __░░░░__
2 ██░░____
3 ____██__
4 ██░░____

Player 2 flips row 1.
  A B C D 
1 __████__
2 ██░░____
3 ____██__
4 ██░░____

Player 1 inserts tile ░░ from the bottom of column B.
  A B C D 
1 __░░██__
2 ██______
3 __░░██__
4 ██░░____

Player 2 inserts tile ██ from the top of column D.
  A B C D 
1 __░░████
2 ██______
3 __░░██__
4 ██░░____

Play